In [31]:
import numpy    
import scipy
import pymeshlab
import radarsimpy
print('`RadarSimPy` used in this example is version: ' +
      str(radarsimpy.__version__))


`RadarSimPy` used in this example is version: 11.3.0


# FMCW Radar

[![Documentations](https://img.shields.io/github/v/tag/radarsimx/radarsimpy?label=Documentation&logo=read-the-docs)](https://radarsimx.github.io/radarsimpy/)
[![Download](https://img.shields.io/github/v/tag/radarsimx/radarsimpy?label=Download&logo=python)](https://radarsimx.com/product/radarsimpy/)

## Introduction

An FMCW (Frequency-Modulated Continuous Wave) radar is a type of radar system that uses frequency modulation to continuously transmit a signal with varying frequency. FMCW radar systems are commonly used in various applications, including automotive radar, weather monitoring, navigation, and industrial sensing. They offer several advantages over other radar types, such as pulsed radar.

Here's how FMCW radar works:

1. **Frequency Modulation**: In an FMCW radar, the radar transmitter emits a continuous waveform that changes frequency over time. This waveform is typically a sinusoidal signal. The frequency change occurs in a linear or ramp-like manner, hence the name "Frequency-Modulated Continuous Wave."

2. **Transmit and Receive Phases**: The radar emits the frequency-modulated signal, which travels towards the target. When the signal encounters an object, a portion of it gets reflected back towards the radar receiver.

3. **Mixing and Beat Frequency**: The reflected signal is mixed with the original transmitted signal in the radar receiver. Because the received signal has traveled a certain distance to the target and back, it experiences a frequency shift due to the Doppler effect and the frequency modulation of the transmitted signal. This frequency shift creates a beat frequency that is proportional to the distance and relative velocity of the target.

4. **Signal Processing**: By analyzing the beat frequency, the radar system can determine both the range (distance) and velocity of the target. The range is determined by the phase shift of the beat frequency, and the velocity is derived from the rate of change of the beat frequency over time.

Advantages of FMCW radar include:

1. **Continuous Operation**: FMCW radars operate continuously without interruptions, making them well-suited for tracking moving targets and providing real-time data.

2. **Range and Velocity Information**: FMCW radar can simultaneously provide information about the range and velocity of targets, making it useful for applications such as automotive collision avoidance and speed measurement.

3. **High Range Resolution**: FMCW radar systems can achieve high range resolution due to the continuous waveform and the use of frequency modulation.

4. **Simplicity and Cost-Effectiveness**: FMCW radar systems can be simpler and more cost-effective to implement compared to some other radar types, like pulsed radar.

5. **Adaptability to Multiple Targets**: FMCW radars can handle multiple targets within their field of view due to their continuous operation and the ability to separate different targets based on their individual beat frequencies.

Despite their advantages, FMCW radars also have limitations, such as reduced performance in certain cluttered environments and susceptibility to interference from other radar systems using similar frequencies. However, advancements in signal processing and technology have helped mitigate many of these challenges.

This illustration showcases a simulation of an FMCW radar using the [`RadarSimPy`](https://radarsimx.com/radarsimx/radarsimpy/) framework. It additionally provides a demonstration of fundamental range and Doppler processing techniques for extracting target range and velocity information.

## Create Radar Model

Firstly, import the required modules from `radarsimpy`. `numpy` will also be needed in this example.


In [32]:
import numpy as np
from radarsimpy import Radar, Transmitter, Receiver


### Transmitter

Setup the basic transmitter parameters through [Transmitter](https://radarsimx.github.io/radarsimpy/radar.html#radarsimpy-transmitter) module.

#### Define Antenna patterns

In this context, we are defining the radiation patterns for both the transmitter and receiver antennas. Please note that idealized patterns are employed for this purpose, but users have the flexibility to incorporate more realistic antenna patterns into the simulation as needed.


In [33]:
az_angle = np.arange(-80, 81, 1)
az_pattern = 20 * np.log10(np.cos(az_angle / 180 * np.pi)**4) + 6

el_angle = np.arange(-80, 81, 1)
el_pattern = 20 * np.log10((np.cos(el_angle / 180 * np.pi))**20) + 6


In [34]:
# H-Plane is Horizontal
# E-Plane is Verticle  

import pandas as pd

# Define the file paths for your CSV files
e_plane_file = "data/E-Plane.csv"
h_plane_file = "data/H-Plane.csv"

# Use pandas to read the CSV data into dataframes
e_plane_df = pd.read_csv(e_plane_file)
h_plane_df = pd.read_csv(h_plane_file)

# You can now work with the dataframes, which are more versatile than NumPy arrays
# For example, you can easily access columns by their headers
el_angle = e_plane_df['Angle']
el_pattern = e_plane_df['Realized Gain']
# e_plane_df['Smooth Realized Gain'] = e_plane_df['Realized Gain'].rolling(window=5).mean()

az_angle = h_plane_df['Angle']
az_pattern = h_plane_df['Realized Gain']

# # can print the first 5 rows of each dataframe
# print("E-Plane Data:")
# print(e_plane_df.head())

# print("\nH-Plane Data:")
# print(h_plane_df.head())


_Plot the antenna patterns_


In [35]:
import plotly.graph_objs as go

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=el_angle,
    y=el_pattern,
    name='Elevation',
))

fig.add_trace(go.Scatter(
    x=az_angle,
    y=az_pattern,
    name='Azimuth',
))

fig.update_layout(
    title='Antenna Pattern',
    yaxis=dict(title='Amplitude (dB)', range=[-30, 30]),
    xaxis=dict(title='Angle (deg)'),
)

fig.show()

#### Define a Transmitter Channel

Here, the location and the radiation patterns of the transmitter channel are defined. All the other properties, such as `polarization` and `modulation`, etc, are using default values as listed in the [documentation](https://radarsimx.github.io/radarsimpy/radar.html#radarsimpy-transmitter).


In [36]:
tx_channel_1 = dict(
    location=(0, 0, 0),
    azimuth_angle=az_angle,
    azimuth_pattern=az_pattern,
    elevation_angle=el_angle,
    elevation_pattern=el_pattern,
)
tx_channel_2 = dict(
    location=(-0.018654, 0, 0),
    azimuth_angle=az_angle,
    azimuth_pattern=az_pattern,
    elevation_angle=el_angle,
    elevation_pattern=el_pattern,
)

#### Define a Radar Transmitter

As shown in the diagram below, `f` and `t` are used to define the waveform modulation. For a linear frequency-modulated continuous waveform (FMCW), it can be specified as `f = [fstart, fend]` and `t = [tstart, tend]`. If `t` is a single number `t = t0`, which is equivalent to `t = [0, t0]`. The bandwidth of the FMCW is `abs(fstart - fend)`. `prp` is the pulse repetition period, and `prp >= (tend - tstart)`.

```
|                       prp
|                  +-----------+
|
|            +---f[1]--->  /            /            /
|                         /            /            /
|                        /            /            /
|                       /            /            /
|                      /            /            /     ...
|                     /            /            /
|                    /            /            /
|                   /            /            /
|      +---f[0]--->/            /            /
|
|                  +-------+
|                 t[0]    t[1]
```


In [37]:
tx = Transmitter(f=[24.010e9, 24.241e9],
                 t=260e-6,
                 tx_power=8,
                 prp=280e-6,
                 pulses=256,
                 channels=[tx_channel_1, tx_channel_2])


### Receiver

Setup the receiver parameters through [Receiver](https://radarsimx.github.io/radarsimpy/radar.html#radarsimpy-receiver) module.

#### Define a Receiver Channel

Similar to the transmitter channel, the location and radiation patterns of the receiver channel are defined. All the other properties, such as `polarization`, `antenna patterns`, are using default values as listed in the [documentation](https://radarsimx.github.io/radarsimpy/radar.html#radarsimpy-receiver).


In [38]:
rx_channel_1 = dict(
    location=(0, 0.032014, 0),
    azimuth_angle=az_angle,
    azimuth_pattern=az_pattern,
    elevation_angle=el_angle,
    elevation_pattern=el_pattern,
)
rx_channel_2 = dict(
    location=(0, 0.037231, 0),
    azimuth_angle=az_angle,
    azimuth_pattern=az_pattern,
    elevation_angle=el_angle,
    elevation_pattern=el_pattern,
)
rx_channel_3 = dict(
    location=(0, -0.043449, 0),
    azimuth_angle=az_angle,
    azimuth_pattern=az_pattern,
    elevation_angle=el_angle,
    elevation_pattern=el_pattern,
)
rx_channel_4 = dict(
    location=(0, -0.049666, 0),
    azimuth_angle=az_angle,
    azimuth_pattern=az_pattern,
    elevation_angle=el_angle,
    elevation_pattern=el_pattern,
)

#### Define a Radar Receiver

- `fs` – Sampling rate (sps)
- `noise_figure` – Noise figure (dB)
- `rf_gain` – Total RF gain (dB)
- `load_resistor` – Load resistor to convert power to voltage (Ohm)
- `baseband_gain` – Total baseband gain (dB)


In [39]:
rx = Receiver(fs=1e6,
              noise_figure=10,
              rf_gain=22,
              load_resistor=450,
              baseband_gain=22,
              channels=[rx_channel_1,rx_channel_2,rx_channel_3,rx_channel_4])


### Radar System

Use the defined transmitter and receiver to create the radar system.


In [40]:
radar001 = dict(location = [5,0.5,1], rotation=[0,0,180])
radar002 = dict(location = [1,4,1], rotation=[0,0,90])

In [41]:
print(radar001['location'])

[5, 0.5, 1]


In [42]:
radar = Radar(transmitter=tx, receiver=rx, location = radar002['location'], rotation=radar002['rotation'])

### Targets

The propertities of targets are defined here. There are 3 targets in this simulation. The locations of the targets are defined through $(x, y, z)$ coordinates in meters, and the speeds of the targets are defined trough $(v_x, v_y, v_z)$ in $m/s$. The propertites of the targets also includes radar cross-section (RCS (dBsm)) and phase (degree).


In [43]:
target_1 = dict(location=(5, 0, 1), speed=(-1, 0, 0), rcs=2, phase=0)
target_2 = dict(location=(15, 0, 2), speed=(0, 0, 0), rcs=20, phase=0)

targets = [target_1, target_2]

## Simulate Baseband Signals

Use the `simulator.simc` module to simulate the baseband samples from the defined radar system and targets.

The output baseband data is a dict including the timestamp and baseband. Both of them are 3-D matrix:

`[channels, pulses, ADC samples]`


In [44]:
from radarsimpy.simulator import simc

data = simc(radar, targets, noise=True)
timestamp = data['timestamp']
baseband = data['baseband']

CUDA error: CUDA-capable device(s) is/are busy or unavailable
CUDA error: CUDA-capable device(s) is/are busy or unavailable
CUDA error: CUDA-capable device(s) is/are busy or unavailable
CUDA error: CUDA-capable device(s) is/are busy or unavailable
CUDA error: CUDA-capable device(s) is/are busy or unavailable
CUDA error: CUDA-capable device(s) is/are busy or unavailable
CUDA error: CUDA-capable device(s) is/are busy or unavailable
CUDA error: CUDA-capable device(s) is/are busy or unavailable
CUDA error: CUDA-capable device(s) is/are busy or unavailable
CUDA error: CUDA-capable device(s) is/are busy or unavailable
CUDA error: CUDA-capable device(s) is/are busy or unavailable
CUDA error: CUDA-capable device(s) is/are busy or unavailable
CUDA error: CUDA-capable device(s) is/are busy or unavailable
CUDA error: CUDA-capable device(s) is/are busy or unavailable
CUDA error: CUDA-capable device(s) is/are busy or unavailable
CUDA error: CUDA-capable device(s) is/are busy or unavailable
CUDA err

Simulation WITH range profile/RDM processing over time

In [30]:
from radarsimpy.simulator import simc
from scipy import signal
import radarsimpy.processing as proc
import pandas as pd
import os

# Provided participant labels
Participant_Labels = ['01','02', '03','05', '08','04', '09', '10', '12', '13', '14', '15', '16', '18', '22', '24']
# REDO parts of 02 and 13

# Updated list of pose labels
Pose_Labels = ['CRW2R', 'CRW2L', 'CT2CW', 'FF2MN', 'MNTRR', 'MNTRL']

# List of versions
versions = ['V1', 'V2', 'V3']

# List of transitions
transitions = ['tx1', 'tx2', 'tx3']

# Prepare the range and Doppler axis for cropping and visualization
max_range = (3e8 * radar.receiver.fs *
            radar.transmitter.pulse_length /
            radar.transmitter.bandwidth / 2)
unambiguous_speed = 3e8 / radar.transmitter.prp[0] / \
    radar.transmitter.fc_vect[0] / 2

range_axis = np.linspace(
    0, max_range, radar.samples_per_pulse, endpoint=False)

doppler_axis = np.linspace(
    -unambiguous_speed/2, unambiguous_speed/2, radar.transmitter.pulses)

# Compute the indices corresponding to the desired range and velocity
range_indices = np.where((range_axis >= 0) & (range_axis <= 15))
vel_indices = np.where((doppler_axis >= -5) & (doppler_axis <= 5))

# Define the range and Doppler windows for FFT
range_window = signal.windows.chebwin(radar.samples_per_pulse, at=60)
doppler_window = signal.windows.chebwin(radar.transmitter.pulses, at=60)

# Calculate the range and velocity resolution
c = 3e8  # speed of light
range_res = c / (2 * tx.bandwidth)
vel_res = c / (2 * np.average(tx.f) * radar.transmitter.prp * radar.transmitter.pulses)

missed_csvs = []

# Iterate over each participant, pose, version, and transition
for participant in Participant_Labels:
    for pose in Pose_Labels:
        for version in versions:
            for transition in transitions:
                print(f'Simulating participant {participant}, pose {pose}, version {version}, transition {transition}')
                # Construct the file path
                path = f'/media/dcope/DC_LaCie/Yoga_Study_MOCAP_Data/{participant}/PosVel_csv/{participant}_{pose}_MC_{version}_{transition}.csv'
                
                # Check if the file exists
                if os.path.exists(path):
                    print(f"Found {path.split('/')[-1]}")
                    df = pd.read_csv(path)
                else:
                    print(f"Did NOT find {path.split('/')[-1]}")
                    missed_csvs.append(path)
                    continue
                            
                # Initialize an empty list for mocap data
                mocap_data = []

                # Prefixes from your CSV columns (assuming all body parts have both position and velocity data)
                prefixes = ['Wrist_R', 'Elbow_R', 'Shoulder_R', 'Wrist_L', 'Elbow_L', 'Shoulder_L', 'Upper_Back', 'Lower_Back', 'Chest', 'Belly', 
                            'Hip_R_Ant', 'Hip_L_Ant', 'Hip_R_Post', 'Hip_L_Post', 'Knee_R', 'Knee_L', 'Ankle_R', 'Ankle_L']

                # Iterate through each row in the DataFrame
                for _, row in df.iterrows():
                    timestep_data = []
                    for prefix in prefixes:
                        location = [row[f'{prefix}_pos_X'], row[f'{prefix}_pos_Y'], row[f'{prefix}_pos_Z']]
                        speed = [row[f'{prefix}_vel_X'], row[f'{prefix}_vel_Y'], row[f'{prefix}_vel_Z']]
                        
                        # Assuming rcs is not present in your data and is a constant value (10 for the example). Adjust as needed.
                        rcs = 1  # Adjust this value if needed
                        timestep_data.append({'location': location, 'speed': speed, 'rcs': rcs})
                    
                    mocap_data.append(timestep_data)

                # # To verify, print the first two timesteps
                # for i in range(2):
                #     print(mocap_data[i])
                #     print("\n")

                # Lists to store results for each time instance
                sim_results = []
                range_profiles = []
                range_doppler_maps = []

                # Iterate over each time instance in the mocap data
                for time_instance in mocap_data:
                    # Simulate radar echoes for the current time instance
                    data = simc(radar, time_instance, noise=True)
                    sim_results.append(data)
                    
                    # Process the baseband data with the range FFT
                    baseband = data['baseband']
                    range_profile = proc.range_fft(baseband, range_window)
                    range_profiles.append(range_profile)
                    
                    # Compute the Range-Doppler map
                    rd_map = proc.doppler_fft(range_profile, doppler_window)

                    # Shift the Range-Doppler map along the Doppler axis
                    rd_map_shifted = np.fft.fftshift(rd_map, axes=1)
                    
                    # Filter and save the isolated Range-Doppler data of interest
                    rd_map_cropped = rd_map_shifted[np.ix_(np.arange(rd_map_shifted.shape[0]), vel_indices[0], range_indices[0])]
                    range_doppler_maps.append(rd_map_cropped)


                # Prepare a tensor to store FFT data
                rd_tensor = np.zeros((len(range_doppler_maps), range_doppler_maps[0].shape[1], range_doppler_maps[0].shape[2], len(range_doppler_maps[0])))

                # Iterate over the 8 channels
                for re_tensor_idx in range(range_doppler_maps[0].shape[0]):
                    
                    # Loop over the 300 images that comprise each GIF for each channel
                    for idx, rd_maps in enumerate(range_doppler_maps):
                        
                        # Store in the tensor
                        rd_tensor[idx, :, :, re_tensor_idx] = 20 * np.log10(np.abs(rd_maps[re_tensor_idx, :, :]))

                    # Construct the save path for the range-Doppler tensor
                    save_path = f'/media/dcope/DC_LaCie/Yoga_Simulated_RADAR_Data/{participant}/radar_sim_npy_002/{participant}_{pose}_MC_{version}_{transition}_channel_{re_tensor_idx+1}_002.npy'
                    
                    # Ensure the directory exists
                    os.makedirs(os.path.dirname(save_path), exist_ok=True)
                    
                    # Save the range-Doppler tensor for CNN processing
                    np.save(save_path, rd_tensor)
                print(f"saved {path.split('/')[-1]}")

print(f"we missed {missed_csvs} csvs")

Simulating participant 01, pose CRW2R, version V1, transition tx1
Found 01_CRW2R_MC_V1_tx1.csv


CUDA error: CUDA-capable device(s) is/are busy or unavailable
CUDA error: CUDA-capable device(s) is/are busy or unavailable
CUDA error: CUDA-capable device(s) is/are busy or unavailable
CUDA error: CUDA-capable device(s) is/are busy or unavailable
CUDA error: CUDA-capable device(s) is/are busy or unavailable
CUDA error: CUDA-capable device(s) is/are busy or unavailable
CUDA error: CUDA-capable device(s) is/are busy or unavailable
CUDA error: CUDA-capable device(s) is/are busy or unavailable
CUDA error: CUDA-capable device(s) is/are busy or unavailable
CUDA error: CUDA-capable device(s) is/are busy or unavailable
CUDA error: CUDA-capable device(s) is/are busy or unavailable
CUDA error: CUDA-capable device(s) is/are busy or unavailable
CUDA error: CUDA-capable device(s) is/are busy or unavailable
CUDA error: CUDA-capable device(s) is/are busy or unavailable
CUDA error: CUDA-capable device(s) is/are busy or unavailable
CUDA error: CUDA-capable device(s) is/are busy or unavailable
CUDA err

KeyboardInterrupt: 

Save as GIF

In [38]:
import numpy as np
import matplotlib.pyplot as plt
import imageio
import os

# Parameters
channels = 1  # Assuming you have 8 channels
max_range_cropped = 15
velocity_min, velocity_max = -5, 5

# Directory where the .npy files are saved
data_dir = './RDM_images/'

# List to store image filenames for the GIF
img_files = []

# Iterate over each channel
for channel_idx in range(channels):
    # Load the saved tensor for the current channel
    tensor_path = "/media/dcope/DC_LaCie/Yoga_Study_MOCAP_Data/01/radar_sim_npy/01_CRW2R_MC_V1_tx1_channel_0.npy"
    rd_tensor = np.load(tensor_path)

    # Loop over the timesteps in the tensor to create images for the GIF
    for idx in range(rd_tensor.shape[0]):
        plt.imshow(rd_tensor[idx, :, :, channel_idx], cmap='binary', aspect='auto',
                   extent=[0, max_range_cropped, velocity_max, velocity_min], vmin=-70, vmax=30)
        plt.colorbar(label='Amplitude (dB)')
        plt.xlabel('Range (m)')
        plt.ylabel('Velocity (m/s)')
        plt.title(f'Range Doppler Map - Channel {channel_idx + 1}')

        # Ensure Y-axis (Velocity) is centered at zero
        plt.ylim([velocity_min, velocity_max])

        # Save each frame as a temporary image
        img_filename = f'./RDM_images/tmp_verify_img_channel_{channel_idx+1}_{idx}.png'
        plt.savefig(img_filename)

        # Add to list
        img_files.append(img_filename)

        # Close the current plot to save memory
        plt.close()

    # Once all images for the current channel are saved, create a GIF
    gif_filename = f'./RDM_images/verify_04_CRW2L_MC_V1_tx1_channel_{channel_idx + 1}.gif'
    imageio.mimsave(gif_filename, [imageio.imread(img) for img in img_files], fps=100)

    # Cleanup the temporary images
    for img_file in img_files:
        os.remove(img_file)

    # Clear the list for the next channel
    img_files.clear()

print("Verification GIFs created!")


/tmp/ipykernel_12565/902546945.py:47: DeprecationWarning:

Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.



Verification GIFs created!


_Plot the baseband samples_


In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=timestamp[0, 0, :],
    y=np.real(baseband[0, 0, :]),
    name='I',
))
fig.add_trace(go.Scatter(
    x=timestamp[0, 0, :],
    y=np.imag(baseband[0, 0, :]),
    name='Q',
))

fig.update_layout(
    title='I/Q Baseband Signals',
    yaxis=dict(title='Amplitude (V)'),
    xaxis=dict(title='Time (s)'),
)

fig.show()

## Radar Signal Processing

### Range Profile

With an FMCW radar, the range profile of the targets could be obtained through a simple FFT across the samples of the chirp. This process is also known as the range-FFT.


In [ ]:
from scipy import signal
import radarsimpy.processing as proc

range_window = signal.windows.chebwin(radar.samples_per_pulse, at=60)
range_profile = proc.range_fft(baseband, range_window)

_Plot range profile_


In [ ]:
max_range = (3e8 * radar.receiver.fs *
             radar.transmitter.pulse_length /
             radar.transmitter.bandwidth / 2)

range_axis = np.linspace(
    0, max_range, radar.samples_per_pulse, endpoint=False)

doppler_axis = np.linspace(
    0, radar.transmitter.pulses, radar.transmitter.pulses, endpoint=False)

fig = go.Figure()

fig.add_trace(go.Surface(x=range_axis, y=doppler_axis, z=20 *
              np.log10(np.abs(range_profile[0, :, :])), colorscale='Rainbow'))

fig.update_layout(
    title='Range Profile',
    height=600,
    scene=dict(
        xaxis=dict(title='Range (m)'),
        yaxis=dict(title='Chirp'),
        zaxis=dict(title='Amplitude (dB)'),
        aspectmode='cube',
    ),
)

fig.show()

### Range-Doppler processing

Perform the FFT across the range profiles could obtan the Doppler information from the data, which is also known as the Doppler FFT.


In [ ]:
doppler_window = signal.windows.chebwin(radar.transmitter.pulses, at=60)
range_doppler = proc.doppler_fft(range_profile, doppler_window)

_Plot range-Doppler map_


In [ ]:
max_range = (3e8 * radar.receiver.fs *
             radar.transmitter.pulse_length /
             radar.transmitter.bandwidth / 2)
unambiguous_speed = 3e8 / radar.transmitter.prp[0] / \
    radar.transmitter.fc_vect[0] / 2

range_axis = np.linspace(
    0, max_range, radar.samples_per_pulse, endpoint=False)

doppler_axis = np.linspace(
    -unambiguous_speed, 0, radar.transmitter.pulses, endpoint=False)

fig = go.Figure()
fig.add_trace(go.Surface(x=range_axis, y=doppler_axis, z=20 *
              np.log10(np.abs(range_doppler[0, :, :])), colorscale='Rainbow'))

fig.update_layout(
    title='Range Doppler',
    height=600,
    scene=dict(
        xaxis=dict(title='Range (m)'),
        yaxis=dict(title='Velocity (m/s)'),
        zaxis=dict(title='Amplitude (dB)'),
        aspectmode='cube',
    ),
)

fig.show()